<a href="https://colab.research.google.com/github/yusuku/DepthEstimationFrom360/blob/main/Metric3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget https://huggingface.co/onnx-community/metric3d-vit-small/resolve/main/onnx/model.onnx

--2024-12-15 15:54:22--  https://huggingface.co/onnx-community/metric3d-vit-small/resolve/main/onnx/model.onnx
Resolving huggingface.co (huggingface.co)... 3.169.137.5, 3.169.137.111, 3.169.137.119, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.5|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/c3/4d/c34d15fb6887deb25d57e54a1619d28b77a2af2aba07fc8a6315751c20120e27/674a665052f01bb2b64687200182f3380f0a462f4b6ef2e51fefd06c84d8ee75?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.onnx%3B+filename%3D%22model.onnx%22%3B&Expires=1734537262&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDUzNzI2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2MzLzRkL2MzNGQxNWZiNjg4N2RlYjI1ZDU3ZTU0YTE2MTlkMjhiNzdhMmFmMmFiYTA3ZmM4YTYzMTU3NTFjMjAxMjBlMjcvNjc0YTY2NTA1MmYwMWJiMmI2NDY4NzIwMDE4MmYzMzgwZjBhNDYyZjRiNmVmMmU1MWZlZmQwNmM4NGQ4ZWU3NT9yZXNwb25zZS1jb250ZW

In [2]:
!pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00


In [8]:
import onnxruntime as ort
import requests
import numpy as np
from PIL import Image

# Load session
ort_session = ort.InferenceSession("/content/model.onnx", providers=['CPUExecutionProvider'])

# Load image
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open("/content/001ad2ad14234e06b2d996d71bb96fc4_color.png").convert("RGB")

# Predict depth
input = np.array(image).transpose(2, 0, 1)
input = np.expand_dims(input, 0) # Add batch dim
onnxruntime_input = {'pixel_values': input.astype(np.float32)}
pred_depth, pred_normal, normal_confidence = ort_session.run(None, onnxruntime_input)

In [9]:
min_val = pred_depth.min()
max_val = pred_depth.max()
normalized = 255 * ((pred_depth - min_val)/(max_val-min_val))
Image.fromarray(normalized[0].astype(np.uint8)).save('depth.png')

In [10]:
from PIL import Image
import torchvision.transforms as transforms
import torch

# 画像ファイルを読み込む
image_path = "/content/f98f146eba5a102cb3441b0963f3c57e.png"  # PNG ファイルのパス
image = Image.open("/content/001ad2ad14234e06b2d996d71bb96fc4_color.png").convert("RGB")

# 変換: PIL Image -> PyTorch Tensor
transform = transforms.Compose([
    transforms.ToTensor()  # Tensor に変換 (値は [0, 1] 範囲に正規化)
])

tensor_image = transform(image)

# 結果を表示
print(f"Tensor shape: {tensor_image.shape}")  # (C, H, W): チャネル、縦、横
print(f"Tensor dtype: {tensor_image.dtype}")  # データ型

Tensor shape: torch.Size([3, 1024, 2048])
Tensor dtype: torch.float32


In [ ]:
import torch
model = torch.hub.load('yvanyin/metric3d', 'metric3d_vit_small', pretrain=True)
pred_depth, confidence, output_dict = model.inference({'input': tensor_image})
pred_normal = output_dict['prediction_normal'][:, :3, :, :] # only available for Metric3Dv2 i.e., ViT models
normal_confidence = output_dict['prediction_normal'][:, 3, :, :] # see https://arxiv.org/abs/2109.09881 for details

Using cache found in /root/.cache/torch/hub/yvanyin_metric3d_main


ModuleNotFoundError: No module named 'mmcv'

In [ ]:
!pip install mmcv-full
!pip install mmengine

  Using cached mmcv-full-1.7.2.tar.gz (607 kB)
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user
